In [1]:
import os
import numpy as np
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler,Normalizer
import pandas as pd
import sklearn.svm as svm
from imblearn.datasets import make_imbalance
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix, r2_score
from sklearn.metrics import mean_squared_error
import xgboost

def categorize(array):
    #print(array)
    new_array=np.zeros_like(array)
    for i in range(0,array.shape[0]): 
        k=array[i]
        if k>0.33:
            new_array[i,:]=1
        else: 
            new_array[i,:]=0
    return new_array
#df = pd.DataFrame(columns=['Parameter','fold1','fold2','fold3','fol4'])
df = pd.DataFrame(columns=['Parameter','fold1','fold2','fold3','fol4'])
#for i in range(1,80):
for i in range(1,25):

    print(i)
    MSE = np.array('mse')
    Accuracy = np.array('Acc')
    r2_sc = np.array('r2_score')
    
    for fold in range(1,5):
        features = pd.read_csv('ICHFeatures.csv',header=0)
        OS_train = pd.read_excel(r'GroundTruth.xlsx', sheet_name="Fold"+str(fold)+'_Seg',header = 0, dtype=str)
        OS_train["ID"] = OS_train["ID"].str.zfill(3)
        #OS_train.columns = ['ID','OS']
        OS_valid = pd.read_excel(r'GroundTruth.xlsx', sheet_name="Fold"+str(fold)+'_Val',header = 0, dtype=str)
        OS_valid["ID"] = OS_valid["ID"].str.zfill(3)
        #OS_valid.columns = ['ID','OS']
        features['ID']=features['ID'].str.replace('ct1','')
        train = pd.merge(features, OS_train, how='right', on='ID')
        test =  pd.merge(features, OS_valid, how='right', on='ID')
        norm_wihtout = [col for col in train.columns if col not in ['ID','Delta','Class']]
        #norm_valid = [col for col in test.columns if col not in ['ID','GCS','Onset','OS']]
        scaler = StandardScaler()
        train_ss = scaler.fit_transform(train[norm_wihtout])
        test_ss = scaler.transform(test[norm_wihtout])
        train[norm_wihtout] = train_ss
        test[norm_wihtout] = test_ss
        #train = train.assign(norm_train.values = train_ss)
        col_withoutID = [col for col in train.columns if col not in ['ID','Class']]
        ros = RandomOverSampler(random_state=42)
        X_res, y_res = ros.fit_resample(train[col_withoutID], train['Class'].values.astype(float))
        X_withDelta = pd.DataFrame(X_res,columns = col_withoutID)
        train_class = pd.DataFrame(y_res, columns = ['Class'])
        col_withoutDelta = [col for col in X_withDelta.columns if col not in ['Delta']]
        train_X = X_withDelta[col_withoutDelta]
        train_y = X_withDelta["Delta"]
        num_features = i
        estimator = xgboost.XGBRegressor(objective='reg:linear', n_estimators=10, seed=123)
        #print(num_features)
        rfe=RFE(estimator,  n_features_to_select=num_features,step=1)
        rfe.fit(train_X,train_y)
        ranking_RFE=rfe.ranking_
        indices=np.where(ranking_RFE==1)
        indices = list(indices[0])
        data_RFE=train_X.iloc[:,indices]
        valid_RFE = test[col_withoutID].iloc[:,indices]
        #print(data_RFE.columns)
        model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08)
        model.fit(data_RFE, train_y)

        Y_pred=model.predict(valid_RFE).ravel()
        #acc=metrics.accuracy_score(test['Delta'].values,Y_pred)
        #print("accuracy score = "+str(acc)) 
        mse = mean_squared_error(test['Delta'].values, Y_pred)
        MSE = np.append(MSE,mse)
        r2_s = r2_score(test['Delta'].values, Y_pred)
        #print(mse)
        #print(r2_s)
        r2_sc = np.append(r2_sc,r2_s)
    #     con_matrix = confusion_matrix(test['Delta'].values.tolist(),Y_pred.tolist())
    #     TN,FP,FN,TP = con_matrix.ravel()
    #     # Sensitivity, hit rate, recall, or true positive rate
    #     TPR = TP/(TP+FN)
    #     # Specificity or true negative rate
    #     TNR = TN/(TN+FP) 
    #     #Precision
    #     PPV = TP/(TP+FP)
    #     Prec = np.append(Prec,PPV)
    #     Sens = np.append(Sens,TPR)
    #     Spec = np.append(Spec,TNR)
        predictions = categorize(Y_pred.reshape(20,1))
        #print(predictions)
        y_test_class = categorize(pd.to_numeric(test['Delta']).values.reshape(20,1))#.reshape(20,1)
        # evaluate predictions
        accuracy = metrics.accuracy_score(y_test_class, predictions)
        #print("Accuracy: %.2f%%" % (accuracy * 100.0))
        Accuracy = np.append(Accuracy,accuracy)
        #best=metrics.mean_squared_error(y_test*1000, y_pred*1000) 
        #print(best)
    MSE = pd.DataFrame(data = MSE.reshape(1,5),columns = df.columns)
    ACC = pd.DataFrame(data = Accuracy.reshape(1,5),columns = df.columns)
    R2_Score = pd.DataFrame(data = r2_sc.reshape(1,5),columns = df.columns)
    # Spec = pd.DataFrame(data = Spec.reshape(1,5),columns = df.columns)
    # df = df.append(Accuracy)
    df = df.append(MSE)
    df = df.append(ACC)
    df= df.append(R2_Score)
    #del Accuracy
    print(np.average(Accuracy[1:].astype(np.float)))

Using TensorFlow backend.


1


/home/navodini/py3_kernel/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/navodini/py3_kernel/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


TypeError: a float is required

In [ ]:
train_y

In [ ]:
pd.to_numeric(test['Delta']).values.size